<h1 align=center><font size = 8>Segmenting and Clustering Neighborhoods in Toronto, Canada</font></h1>

First of all, we have to import libraries and the dataset

## Step 1: Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 18.7MB/s eta 0:00:01
Libraries imported.


## Step 2: Download dataset

In this case, the dataset that we have is given by Wikipedia, which is a table of neighborhoods and their post code. This table is given in HTML, so we have to convert it to a Panda Dataframe.

To do that, I create a string with the HTML code.

In [44]:
html_string = """
<table class="wikitable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East York
</td>
<td>Parkview Hill / Woodbine Gardens
</td></tr>
<tr>
<td>M5B
</td>
<td>Downtown Toronto
</td>
<td>Garden District, Ryerson
</td></tr>
<tr>
<td>M6B
</td>
<td>North York
</td>
<td>Glencairn
</td></tr>
<tr>
<td>M7B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9B
</td>
<td>Etobicoke
</td>
<td>West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
</td></tr>
<tr>
<td>M1C
</td>
<td>Scarborough
</td>
<td>Rouge Hill / Port Union / Highland Creek
</td></tr>
<tr>
<td>M2C
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3C
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4C
</td>
<td>East York
</td>
<td>Woodbine Heights
</td></tr>
<tr>
<td>M5C
</td>
<td>Downtown Toronto
</td>
<td>St. James Town
</td></tr>
<tr>
<td>M6C
</td>
<td>York
</td>
<td>Humewood-Cedarvale
</td></tr>
<tr>
<td>M7C
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8C
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9C
</td>
<td>Etobicoke
</td>
<td>Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
</td></tr>
<tr>
<td>M1E
</td>
<td>Scarborough
</td>
<td>Guildwood / Morningside / West Hill
</td></tr>
<tr>
<td>M2E
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3E
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4E
</td>
<td>East Toronto
</td>
<td>The Beaches
</td></tr>
<tr>
<td>M5E
</td>
<td>Downtown Toronto
</td>
<td>Berczy Park
</td></tr>
<tr>
<td>M6E
</td>
<td>York
</td>
<td>Caledonia-Fairbanks
</td></tr>
<tr>
<td>M7E
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8E
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9E
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1G
</td>
<td>Scarborough
</td>
<td>Woburn
</td></tr>
<tr>
<td>M2G
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3G
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4G
</td>
<td>East York
</td>
<td>Leaside
</td></tr>
<tr>
<td>M5G
</td>
<td>Downtown Toronto
</td>
<td>Central Bay Street
</td></tr>
<tr>
<td>M6G
</td>
<td>Downtown Toronto
</td>
<td>Christie
</td></tr>
<tr>
<td>M7G
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8G
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9G
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1H
</td>
<td>Scarborough
</td>
<td>Cedarbrae
</td></tr>
<tr>
<td>M2H
</td>
<td>North York
</td>
<td>Hillcrest Village
</td></tr>
<tr>
<td>M3H
</td>
<td>North York
</td>
<td>Bathurst Manor / Wilson Heights / Downsview North
</td></tr>
<tr>
<td>M4H
</td>
<td>East York
</td>
<td>Thorncliffe Park
</td></tr>
<tr>
<td>M5H
</td>
<td>Downtown Toronto
</td>
<td>Richmond / Adelaide / King
</td></tr>
<tr>
<td>M6H
</td>
<td>West Toronto
</td>
<td>Dufferin / Dovercourt Village
</td></tr>
<tr>
<td>M7H
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8H
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9H
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1J
</td>
<td>Scarborough
</td>
<td>Scarborough Village
</td></tr>
<tr>
<td>M2J
</td>
<td>North York
</td>
<td>Fairview / Henry Farm / Oriole
</td></tr>
<tr>
<td>M3J
</td>
<td>North York
</td>
<td>Northwood Park / York University
</td></tr>
<tr>
<td>M4J
</td>
<td>East York
</td>
<td>East Toronto
</td></tr>
<tr>
<td>M5J
</td>
<td>Downtown Toronto
</td>
<td>Harbourfront East / Union Station / Toronto Islands
</td></tr>
<tr>
<td>M6J
</td>
<td>West Toronto
</td>
<td>Little Portugal / Trinity
</td></tr>
<tr>
<td>M7J
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8J
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9J
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1K
</td>
<td>Scarborough
</td>
<td>Kennedy Park / Ionview / East Birchmount Park
</td></tr>
<tr>
<td>M2K
</td>
<td>North York
</td>
<td>Bayview Village
</td></tr>
<tr>
<td>M3K
</td>
<td>North York
</td>
<td>Downsview
</td></tr>
<tr>
<td>M4K
</td>
<td>East Toronto
</td>
<td>The Danforth West / Riverdale
</td></tr>
<tr>
<td>M5K
</td>
<td>Downtown Toronto
</td>
<td>Toronto Dominion Centre / Design Exchange
</td></tr>
<tr>
<td>M6K
</td>
<td>West Toronto
</td>
<td>Brockton / Parkdale Village / Exhibition Place
</td></tr>
<tr>
<td>M7K
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8K
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9K
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1L
</td>
<td>Scarborough
</td>
<td>Golden Mile / Clairlea / Oakridge
</td></tr>
<tr>
<td>M2L
</td>
<td>North York
</td>
<td>York Mills / Silver Hills
</td></tr>
<tr>
<td>M3L
</td>
<td>North York
</td>
<td>Downsview
</td></tr>
<tr>
<td>M4L
</td>
<td>East Toronto
</td>
<td>India Bazaar / The Beaches West
</td></tr>
<tr>
<td>M5L
</td>
<td>Downtown Toronto
</td>
<td>Commerce Court / Victoria Hotel
</td></tr>
<tr>
<td>M6L
</td>
<td>North York
</td>
<td>North Park / Maple Leaf Park / Upwood Park
</td></tr>
<tr>
<td>M7L
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8L
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9L
</td>
<td>North York
</td>
<td>Humber Summit
</td></tr>
<tr>
<td>M1M
</td>
<td>Scarborough
</td>
<td>Cliffside / Cliffcrest / Scarborough Village West
</td></tr>
<tr>
<td>M2M
</td>
<td>North York
</td>
<td>Willowdale / Newtonbrook
</td></tr>
<tr>
<td>M3M
</td>
<td>North York
</td>
<td>Downsview
</td></tr>
<tr>
<td>M4M
</td>
<td>East Toronto
</td>
<td>Studio District
</td></tr>
<tr>
<td>M5M
</td>
<td>North York
</td>
<td>Bedford Park / Lawrence Manor East
</td></tr>
<tr>
<td>M6M
</td>
<td>York
</td>
<td>Del Ray / Mount Dennis / Keelsdale and Silverthorn
</td></tr>
<tr>
<td>M7M
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8M
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9M
</td>
<td>North York
</td>
<td>Humberlea / Emery
</td></tr>
<tr>
<td>M1N
</td>
<td>Scarborough
</td>
<td>Birch Cliff / Cliffside West
</td></tr>
<tr>
<td>M2N
</td>
<td>North York
</td>
<td>Willowdale
</td></tr>
<tr>
<td>M3N
</td>
<td>North York
</td>
<td>Downsview
</td></tr>
<tr>
<td>M4N
</td>
<td>Central Toronto
</td>
<td>Lawrence Park
</td></tr>
<tr>
<td>M5N
</td>
<td>Central Toronto
</td>
<td>Roselawn
</td></tr>
<tr>
<td>M6N
</td>
<td>York
</td>
<td>Runnymede / The Junction North
</td></tr>
<tr>
<td>M7N
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8N
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9N
</td>
<td>York
</td>
<td>Weston
</td></tr>
<tr>
<td>M1P
</td>
<td>Scarborough
</td>
<td>Dorset Park / Wexford Heights / Scarborough Town Centre
</td></tr>
<tr>
<td>M2P
</td>
<td>North York
</td>
<td>York Mills West
</td></tr>
<tr>
<td>M3P
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4P
</td>
<td>Central Toronto
</td>
<td>Davisville North
</td></tr>
<tr>
<td>M5P
</td>
<td>Central Toronto
</td>
<td>Forest Hill North &amp; West
</td></tr>
<tr>
<td>M6P
</td>
<td>West Toronto
</td>
<td>High Park / The Junction South
</td></tr>
<tr>
<td>M7P
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8P
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9P
</td>
<td>Etobicoke
</td>
<td>Westmount
</td></tr>
<tr>
<td>M1R
</td>
<td>Scarborough
</td>
<td>Wexford / Maryvale
</td></tr>
<tr>
<td>M2R
</td>
<td>North York
</td>
<td>Willowdale
</td></tr>
<tr>
<td>M3R
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4R
</td>
<td>Central Toronto
</td>
<td>North Toronto West
</td></tr>
<tr>
<td>M5R
</td>
<td>Central Toronto
</td>
<td>The Annex / North Midtown / Yorkville
</td></tr>
<tr>
<td>M6R
</td>
<td>West Toronto
</td>
<td>Parkdale / Roncesvalles
</td></tr>
<tr>
<td>M7R
</td>
<td>Mississauga
</td>
<td>Canada Post Gateway Processing Centre
</td></tr>
<tr>
<td>M8R
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9R
</td>
<td>Etobicoke
</td>
<td>Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens
</td></tr>
<tr>
<td>M1S
</td>
<td>Scarborough
</td>
<td>Agincourt
</td></tr>
<tr>
<td>M2S
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3S
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4S
</td>
<td>Central Toronto
</td>
<td>Davisville
</td></tr>
<tr>
<td>M5S
</td>
<td>Downtown Toronto
</td>
<td>University of Toronto / Harbord
</td></tr>
<tr>
<td>M6S
</td>
<td>West Toronto
</td>
<td>Runnymede / Swansea
</td></tr>
<tr>
<td>M7S
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8S
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9S
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1T
</td>
<td>Scarborough
</td>
<td>Clarks Corners / Tam O'Shanter / Sullivan
</td></tr>
<tr>
<td>M2T
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3T
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4T
</td>
<td>Central Toronto
</td>
<td>Moore Park / Summerhill East
</td></tr>
<tr>
<td>M5T
</td>
<td>Downtown Toronto
</td>
<td>Kensington Market / Chinatown / Grange Park
</td></tr>
<tr>
<td>M6T
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M7T
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8T
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9T
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1V
</td>
<td>Scarborough
</td>
<td>Milliken / Agincourt North / Steeles East / L'Amoreaux East
</td></tr>
<tr>
<td>M2V
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3V
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4V
</td>
<td>Central Toronto
</td>
<td>Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
</td></tr>
<tr>
<td>M5V
</td>
<td>Downtown Toronto
</td>
<td>CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst
<pre> Quay / South Niagara / Island airport
</pre>
</td></tr>
<tr>
<td>M6V
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M7V
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8V
</td>
<td>Etobicoke
</td>
<td>New Toronto / Mimico South / Humber Bay Shores
</td></tr>
<tr>
<td>M9V
</td>
<td>Etobicoke
</td>
<td>South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens
</td></tr>
<tr>
<td>M1W
</td>
<td>Scarborough
</td>
<td>Steeles West / L'Amoreaux West
</td></tr>
<tr>
<td>M2W
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3W
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4W
</td>
<td>Downtown Toronto
</td>
<td>Rosedale
</td></tr>
<tr>
<td>M5W
</td>
<td>Downtown Toronto
</td>
<td>Stn A PO Boxes
</td></tr>
<tr>
<td>M6W
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M7W
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8W
</td>
<td>Etobicoke
</td>
<td>Alderwood / Long Branch
</td></tr>
<tr>
<td>M9W
</td>
<td>Etobicoke
</td>
<td>Northwest
</td></tr>
<tr>
<td>M1X
</td>
<td>Scarborough
</td>
<td>Upper Rouge
</td></tr>
<tr>
<td>M2X
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3X
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4X
</td>
<td>Downtown Toronto
</td>
<td>St. James Town / Cabbagetown
</td></tr>
<tr>
<td>M5X
</td>
<td>Downtown Toronto
</td>
<td>First Canadian Place / Underground city
</td></tr>
<tr>
<td>M6X
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M7X
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8X
</td>
<td>Etobicoke
</td>
<td>The Kingsway / Montgomery Road  / Old Mill North
</td></tr>
<tr>
<td>M9X
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1Y
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2Y
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3Y
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4Y
</td>
<td>Downtown Toronto
</td>
<td>Church and Wellesley
</td></tr>
<tr>
<td>M5Y
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M6Y
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M7Y
</td>
<td>East Toronto
</td>
<td>Business reply mail Processing CentrE
</td></tr>
<tr>
<td>M8Y
</td>
<td>Etobicoke
</td>
<td>Old Mill South / King's Mill Park / Sunnylea / Humber Bay / Mimico NE / The Queensway East / Royal York South East / Kingsway Park South East
</td></tr>
<tr>
<td>M9Y
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M1Z
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2Z
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3Z
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M4Z
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M5Z
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M6Z
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M7Z
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M8Z
</td>
<td>Etobicoke
</td>
<td>Mimico NW / The Queensway West / South of Bloor / Kingsway Park South West / Royal York South West
</td></tr>
<tr>
<td>M9Z
</td>
<td>Not assigned
</td>
<td>
</td></tr></tbody></table>

"""

Then, a .read_html function is used to convert it into a panda dataframe. Dataframe is sorted by postal code

In [48]:
df_can = pd.read_html(html_string)
df_can = df_can[0].sort_values(by = ["Postal code"])
df_can.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
9,M1B,Scarborough,Malvern / Rouge
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
27,M1E,Scarborough,Guildwood / Morningside / West Hill
36,M1G,Scarborough,Woburn


In [49]:
df_can.dropna(inplace = True)
df_can.drop(df_can.loc[df_can['Borough']=='Not assigned'].index, inplace=True)
df_can.head()

,Postal code,Borough,Neighborhood
9,M1B,Scarborough,Malvern / Rouge
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
27,M1E,Scarborough,Guildwood / Morningside / West Hill
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


In [50]:
df_can.sort_values(by=["Postal code"],axis=0)
df_can = df_can.reset_index(drop=True)

Another criteria is to group the neighborhoods by post code, which is already done in the wikipedia table given. The problem is that is separated in each row with a slash "/", so there is need to replace it with a comma " ,". Then, a reset index is done.

In [51]:
df_can['Neighborhood'] = df_can['Neighborhood'].str.replace(" /",",")
df_can.reset_index(drop=True, inplace = True)
df_can.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


And there we have it. The dataframe is sorted by Postal code and every neighborhood is in the row of each postal code. Let's see the shape of the dataframe:

In [47]:
df_can.shape

(180, 3)